In [ ]:
!pip install feedparser

### Title Changes

In [ ]:
import feedparser
from newspaper import Article
from datetime import datetime

def extract_google_news_rss(query="latest", language="en"):
    query = query.replace(" ", "+")
    rss_url = f"https://news.google.com/rss/search?q={query}&hl={language}&gl=US&ceid=US:{language}"
    
    feed = feedparser.parse(rss_url)
    articles = []

    for entry in feed.entries:
        try:
            # Extract pub date
            pub_date = datetime(*entry.published_parsed[:6]) if "published_parsed" in entry else datetime.now()
            formatted_date = pub_date.strftime("%Y-%m-%d")
            formatted_time = pub_date.strftime("%H:%M:%S")

            # Clean source
            source = entry.source.title if "source" in entry else "Unknown"

            # Fallback values
            title = entry.title
            summary = entry.summary if "summary" in entry else ""

            # Try to get better content from the real article
            article = Article(entry.link)
            article.download()
            article.parse()

            if article.title and article.title.lower() != "google news":
                title = article.title
            if article.text:
                summary = article.text[:300]

            articles.append({
                "title": title,
                "description": summary.strip(),
                "source": source,
                "date": formatted_date,
                "time": formatted_time,
                "url": entry.link
            })

        except Exception as e:
            print(f"Error processing article: {entry.link} — {e}")
            continue

    return articles

# ✅ Example usage
if __name__ == "__main__":
    news_data = extract_google_news_rss("AI news")
    for news in news_data[:5]:
        print(f"Title: {news['title']}")
        print(f"Description: {news['description']}")
        print(f"Source: {news['source']}")
        print(f"Date: {news['date']}")
        print(f"Time: {news['time']}")
        print(f"URL: {news['url']}")
        print("-" * 100)


### Description Extraction

In [ ]:
import feedparser
from newspaper import Article
from newspaper import Config
from datetime import datetime
import requests
from urllib.parse import urlparse
import time

def resolve_google_redirect(google_url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
        }
        response = requests.get(google_url, headers=headers, timeout=10, allow_redirects=True)
        return response.url
    except Exception as e:
        print(f"⚠️ Failed to resolve redirect: {google_url} — {e}")
        return None

def extract_google_news(query="AI", language="en"):
    query = query.replace(" ", "+")
    rss_url = f"https://news.google.com/rss/search?q={query}&hl={language}&gl=US&ceid=US:{language}"

    feed = feedparser.parse(rss_url)
    articles = []

    config = Config()
    config.browser_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    config.request_timeout = 10

    for entry in feed.entries:
        try:
            title = entry.title
            source = entry.source.title if "source" in entry else "Unknown"
            date_published = datetime(*entry.published_parsed[:6])
            formatted_date = date_published.strftime("%Y-%m-%d")
            formatted_time = date_published.strftime("%H:%M:%S")

            google_link = entry.link

            # 🔁 Resolve to real URL
            real_url = resolve_google_redirect(google_link)
            if not real_url or "news.google.com" in urlparse(real_url).netloc:
                print(f"⚠️ Skipping unresolved: {google_link}")
                continue

            # 🔍 Extract content using newspaper
            article = Article(real_url, config=config)
            article.download()
            article.parse()

            if article.title and article.title.lower() != "google news":
                title = article.title

            text = article.text.strip() if article.text else "⚠️ No content"

            articles.append({
                "title": title,
                "description": text,
                "source": source,
                "date": formatted_date,
                "time": formatted_time,
                "url": real_url
            })

            # optional sleep to avoid being rate-limited
            time.sleep(1)

        except Exception as e:
            print(f"❌ Error with article: {entry.link} — {e}")
            continue

    return articles

# ✅ Run it
if __name__ == "__main__":
    results = extract_google_news("AI technology")
    for news in results[:5]:
        print(f"Title: {news['title']}")
        print(f"Source: {news['source']}")
        print(f"Date: {news['date']} | Time: {news['time']}")
        print(f"URL: {news['url']}")
        print(f"\nFull Description:\n{news['description']}\n")
        print("=" * 120)

In [31]:
import feedparser
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import trafilatura

def resolve_google_redirect(google_url):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(google_url, headers=headers, timeout=10, allow_redirects=True)
        return response.url
    except Exception as e:
        print(f"⚠️ Redirect fail: {e}")
        return None

def extract_google_news_trafilatura(query="AI news", language="en"):
    query = query.replace(" ", "+")
    rss_url = f"https://news.google.com/rss/search?q={query}&hl={language}&gl=US&ceid=US:{language}"

    feed = feedparser.parse(rss_url)
    articles = []

    for entry in feed.entries:
        try:
            title = entry.title
            source = entry.source.title if "source" in entry else "Unknown"
            pub_date = datetime(*entry.published_parsed[:6])
            formatted_date = pub_date.strftime("%Y-%m-%d")
            formatted_time = pub_date.strftime("%H:%M:%S")

            google_link = entry.link
            real_url = resolve_google_redirect(google_link)
            if not real_url or "news.google.com" in real_url:
                continue

            headers = {"User-Agent": "Mozilla/5.0"}
            resp = requests.get(real_url, headers=headers, timeout=10)
            downloaded = trafilatura.extract(resp.text)

            full_text = downloaded.strip() if downloaded else "⚠️ No content extracted."

            articles.append({
                "title": title,
                "description": full_text,
                "source": source,
                "date": formatted_date,
                "time": formatted_time,
                "url": real_url
            })

        except Exception as e:
            print(f"❌ Error: {e}")
            continue

    return articles

# ✅ Example run
if __name__ == "__main__":
    results = extract_google_news_trafilatura("AI chip ban China")
    for news in results[:3]:
        print(f"Title: {news['title']}")
        print(f"Source: {news['source']}")
        print(f"Date: {news['date']} | Time: {news['time']}")
        print(f"URL: {news['url']}")
        print(f"\nFull Description:\n{news['description']}")
        print("=" * 120)


In [37]:
import feedparser
import requests
from bs4 import BeautifulSoup
from readability import Document
from datetime import datetime

def extract_real_url_from_summary(summary_html):
    soup = BeautifulSoup(summary_html, "html.parser")
    a_tag = soup.find("a")
    return a_tag['href'] if a_tag else None

def extract_google_news_articles(query="AI technology"):
    query = query.replace(" ", "+")
    rss_url = f"https://news.google.com/rss/search?q={query}&hl=en&gl=US&ceid=US:en"
    feed = feedparser.parse(rss_url)

    headers = {"User-Agent": "Mozilla/5.0"}
    articles = []

    for entry in feed.entries:
        try:
            title = entry.title
            source = entry.source.title if "source" in entry else "Unknown"
            pub_date = datetime(*entry.published_parsed[:6])
            date = pub_date.strftime("%Y-%m-%d")
            time = pub_date.strftime("%H:%M:%S")

            real_url = extract_real_url_from_summary(entry.summary)
            if not real_url:
                continue

            # 🧠 Fetch HTML and extract full content using readability
            response = requests.get(real_url, headers=headers, timeout=10)
            doc = Document(response.text)
            full_text_html = doc.summary()
            full_text = BeautifulSoup(full_text_html, "html.parser").get_text(separator="\n").strip()

            if not full_text:
                full_text = "⚠️ No content extracted."

            articles.append({
                "title": title,
                "description": full_text,
                "source": source,
                "date": date,
                "time": time,
                "url": real_url
            })

        except Exception as e:
            print(f"❌ Failed to extract: {e}")
            continue

    return articles

# 🚀 Use It
if __name__ == "__main__":
    news = extract_google_news_articles("AI chip China")
    for a in news[:3]:
        print(f"Title: {a['title']}")
        print(f"Source: {a['source']}")
        print(f"Date: {a['date']} | Time: {a['time']}")
        print(f"URL: {a['url']}")
        print(f"\nFull Description:\n{a['description']}")
        print("=" * 100)


Title: Nvidia says it will restart sales of a key AI chip to China, in a reversal of US restrictions - CNN
Source: CNN
Date: 2025-07-15 | Time: 17:41:00
URL: https://news.google.com/rss/articles/CBMijwFBVV95cUxNMUJtemdGTzdFNGxfVlE1TURscVBCd3Rzb0xNSkhqNnVYc0I3N1YxOG13QVZReXlSZEZqdmJHMExMLW9ZeEpQRDR2eE9TSnp3QWItUTVHOHBwWTlOaTZ3SGxBN3BPZmJIZ2J1RzlhZWdoazk1Z21naU95MnJkOHEtWkZIMU5PZVBGaUloVGZhTdIBlAFBVV95cUxQSGRfOGNoQnVMbFRTV1c5WG1ydWQ3Tnp1Q0htQWJmbFktWUxmMnQxUFZWNS16QTFjd1Y5LUNjclNzc1lZOW5NWjl1S0c1NTl1dE5hcUdwRWU5OXF4aWFUZjE5UXpvM3pWWnlsR1FRcEI2OHM3NmZBQUc5UWMzTEZ0Y3pJMVpmYk9ZNThzYjVJUkw3MTRT?oc=5

Full Description:
⚠️ No content extracted.
Title: Malaysia closes a back door that may have allowed US-sourced AI chips to reach China - theregister.com
Source: theregister.com
Date: 2025-07-15 | Time: 01:02:00
URL: https://news.google.com/rss/articles/CBMiggFBVV95cUxOVlVuSDVaanBBY0gzd1F6UWYweXVQeHJqbGs0S1dqbTBqbmVoSUI5V3UwUW5RTy1LbndsVHBtMEt0alM1eDJQODhJVDYwUlk1Z3ZtdXVRemlQc3VNQVNWQWN5b2Q5ejlld

In [36]:
!pip install readability-lxml beautifulsoup4 feedparser requests